In [1]:
# Install a pip package in the current Jupyter kernel
import sys
sys.path.append('../')
sys.path.append('../wdtools')
sys.path.append('../corv/src')

### General
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from astropy.io import fits
from tqdm import tqdm
import pandas as pd

### Tools
from astropy.io import fits
import wdtools
import data_selector as ds
import corv
import WD_models

#corv.sdss.make_catalogs()

Please install galpy if you want to use the gaia module. otherwise, ignore this.
could not find pickled WD models
using local paths
star and exposure catalogs not found! check paths and run make_catalogs() if you want to use sdss functionality. otherwise ignore.


In [1]:
#ds.select_sdss4()

In [ ]:
###### lines = [6562.79, 4861.35, 4340.47, 4101.73, 3970.07, 3889.06, 3835.40, 5893]

catalog = ds.get_catalog('data/catalog_sdss4.csv')

wd_pos = [1,2,1,1,2,1,2,2,2,2,2,2,2]
ms_pos = [2,1,2,2,1,2,1,1,1,1,1,1,1]

catalog = catalog.drop(8)
catalog.reset_index(inplace=True, drop=True)

#ds.plot_binary_spectra(catalog, lines, 'sdss4')

In [ ]:
from __future__ import print_function
from astropy.coordinates import SkyCoord
import astropy.units as u
from dustmaps.sfd import SFDQuery

coords = SkyCoord('12h30m25.3s', '15d15m58.1s', frame='icrs')
sfd = SFDQuery()
ebv = sfd(coords)

coords = SkyCoord('12h30m25.3s', '15d15m58.1s', frame='icrs')
print('E(B-V) = {:.3f} mag'.format(ebv))

In [ ]:
l = [catalog['l{}'.format(wd_pos[i])][i] for i in range(len(wd_pos))]
b = [catalog['b{}'.format(wd_pos[i])][i] for i in range(len(wd_pos))]

coords = [SkyCoord(frame="galactic", l=l[i]*u.deg, b=b[i]*u.deg) for i in range(len(l))]
catalog['wd_coords'] = coords

In [ ]:
catalog['E(B-V)'] = [sfd(catalog['wd_coords'][i]) for i in range(len(catalog))]

In [ ]:
model = WD_models.load_model('be', 'be', 'o', 'H',
                             HR_bands=('Su-Sg', 'Su'),)

catalog['wd_u-g'] = [catalog['u{}'.format(wd_pos[i])][i] - catalog['g{}'.format(wd_pos[i])][i] for i in range(len(wd_pos))]
catalog['wd_umag'] = [catalog['u{}'.format(wd_pos[i])][i] for i in range(len(wd_pos))]

age_cool = model['HR_to_age_cool'](catalog['u-g'], catalog['u'])

print('bp3_rp3: ', catalog['u-g'], '\nM_G3: ', catalog['u'])
print('cooling ages:', age_cool, 'Gyr')